In [56]:
import pandas as pd
import psycopg2

In [57]:
CONNECT_DB = "host=localhost port=5432 dbname=cloud_admin user=cloud_admin password=cloud_admin"

Create table

In [59]:
create_table_query = '''CREATE TABLE bs140513_032310 (
    step int,
    customer varchar (20),
    age varchar (10),
    gender varchar (10),
    zipcodeOri varchar (20),
    merchant varchar (20),
    zipMerchant varchar (20),
    category varchar (30),
    amount float4,
    fraud boolean
); '''

try:
    # Make connection to db
    cxn = psycopg2.connect(CONNECT_DB)

    # Create a cursor to db
    cur = cxn.cursor()
    
    # Send sql query to request
    cur.execute(create_table_query)
    records = cxn.commit()

except (Exception, psycopg2.Error) as error :
    print ("Error while connecting to PostgreSQL", error)
    
finally:
    #closing database connection.
    if(cxn):
        cur.close()
        cxn.close()
        print("PostgreSQL connection is closed")

print(f'Records:\n {records}')

PostgreSQL connection is closed
Records:
 None


Add data to table

In [60]:
try:
    # Make connection to db
    cxn = psycopg2.connect(CONNECT_DB)
    
    # Create a cursor to db
    cur = cxn.cursor()
    
    with open('./bs140513_032310.csv', 'r') as f:
        # skip first row, header row
        next(f)
        cur.copy_from(f, 'bs140513_032310', sep=",")
        cxn.commit()

except (Exception, psycopg2.Error) as error :
    print ("Error while connecting to PostgreSQL", error)
    
finally:
    #closing database connection.
    if(cxn):
        cur.close()
        cxn.close()
        print("PostgreSQL connection is closed")
        print("bs140513_032310 table populated")


PostgreSQL connection is closed
bs140513_032310 table populated


Select data from server

In [61]:
#selecting data from server using fetchall()
def db_server_fetch(sql_query):
    try:
        # Make connection to db
        cxn = psycopg2.connect(CONNECT_DB)

        # Create a cursor to db
        cur = cxn.cursor()

        # Send sql query to request
        cur.execute(sql_query)
        records = cur.fetchall()

    except (Exception, psycopg2.Error) as error :
        print ("Error while connecting to PostgreSQL", error)

    finally:
        #closing database connection.
        if(cxn):
            cur.close()
            cxn.close()
            print("PostgreSQL connection is closed")
        return records

In [62]:
select_query = '''SELECT * FROM bs140513_032310;'''

records = db_server_fetch(select_query)
#print(records)

PostgreSQL connection is closed


Create dataframe

In [66]:
df = pd.DataFrame(records, columns=["step", "customer", "age", "gender", 
"zipcodeOri", "merchant", "zipMerchant","category","amount","fraud"])

EDA (Exploratory Data Analysis)

In [68]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 594643 entries, 0 to 594642
Data columns (total 10 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   step         594643 non-null  int64  
 1   customer     594643 non-null  object 
 2   age          594643 non-null  object 
 3   gender       594643 non-null  object 
 4   zipcodeOri   594643 non-null  object 
 5   merchant     594643 non-null  object 
 6   zipMerchant  594643 non-null  object 
 7   category     594643 non-null  object 
 8   amount       594643 non-null  float64
 9   fraud        594643 non-null  bool   
dtypes: bool(1), float64(1), int64(1), object(7)
memory usage: 41.4+ MB


In [69]:
df.dtypes

step             int64
customer        object
age             object
gender          object
zipcodeOri      object
merchant        object
zipMerchant     object
category        object
amount         float64
fraud             bool
dtype: object

In [70]:
#Check null
if df.isnull().values.any():
    print("Ada nilai null ")
else:
    print("Tidak ada nilai null")

Tidak ada nilai null


In [73]:
# to show all columns in `.describe` add `include="all"`
df.describe(include="all")

,step,customer,age,gender,zipcodeOri,merchant,zipMerchant,category,amount,fraud
count,594643.000000,594643,594643,594643,594643,594643,594643,594643,594643.000000,594643
unique,NaN,4112,8,4,1,50,1,15,NaN,2
top,NaN,'C1978250683','2','F','28007','M1823072687','28007','es_transportation',NaN,False
freq,NaN,265,187310,324565,594643,299693,594643,505119,NaN,587443
mean,94.986827,NaN,NaN,NaN,NaN,NaN,NaN,NaN,37.890135,NaN
std,51.053632,NaN,NaN,NaN,NaN,NaN,NaN,NaN,111.402831,NaN
min,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,NaN
25%,52.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13.740000,NaN
50%,97.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,26.900000,NaN
75%,139.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,42.540000,NaN


In [76]:
df.describe()

,step,amount
count,594643.000000,594643.000000
mean,94.986827,37.890135
std,51.053632,111.402831
min,0.000000,0.000000
25%,52.000000,13.740000
50%,97.000000,26.900000
75%,139.000000,42.540000
max,179.000000,8329.960000


In [77]:
df.head()

,step,customer,age,gender,zipcodeOri,merchant,zipMerchant,category,amount,fraud
0,0,'C1093826151','4','M','28007','M348934600','28007','es_transportation',4.55,False
1,0,'C352968107','2','M','28007','M348934600','28007','es_transportation',39.68,False
2,0,'C2054744914','4','F','28007','M1823072687','28007','es_transportation',26.89,False
3,0,'C1760612790','3','M','28007','M348934600','28007','es_transportation',17.25,False
4,0,'C757503768','5','M','28007','M348934600','28007','es_transportation',35.72,False


In [78]:
df.tail()

,step,customer,age,gender,zipcodeOri,merchant,zipMerchant,category,amount,fraud
594638,179,'C1753498738','3','F','28007','M1823072687','28007','es_transportation',20.53,False
594639,179,'C650108285','4','F','28007','M1823072687','28007','es_transportation',50.73,False
594640,179,'C123623130','2','F','28007','M349281107','28007','es_fashion',22.44,False
594641,179,'C1499363341','5','M','28007','M1823072687','28007','es_transportation',14.46,False
594642,179,'C616528518','4','F','28007','M1823072687','28007','es_transportation',26.93,False


In [80]:
df.count()

step           594643
customer       594643
age            594643
gender         594643
zipcodeOri     594643
merchant       594643
zipMerchant    594643
category       594643
amount         594643
fraud          594643
dtype: int64

TRANSFORMASI

Clean ('') in the columns customer, age, gender, zipcodeOri, merchant, zipMerchant, category

In [82]:
clean_columns = ['customer', 'age', 'gender', 'zipcodeOri', 'merchant', 'zipMerchant', 'category']
for col in clean_columns:
    df[col] = df[col].str.replace("'", "")

#Show 5 data
df.head()

,step,customer,age,gender,zipcodeOri,merchant,zipMerchant,category,amount,fraud
0,0,C1093826151,4,M,28007,M348934600,28007,es_transportation,4.55,False
1,0,C352968107,2,M,28007,M348934600,28007,es_transportation,39.68,False
2,0,C2054744914,4,F,28007,M1823072687,28007,es_transportation,26.89,False
3,0,C1760612790,3,M,28007,M348934600,28007,es_transportation,17.25,False
4,0,C757503768,5,M,28007,M348934600,28007,es_transportation,35.72,False


Replace U with average age in the age column

In [83]:
#Count average age from valid data
mean_age = df[df['age'] != 'U']['age'].astype(int).mean()

In [84]:
#Replace 'U' with average age
df.loc[df['age'] == 'U', 'age'] = mean_age

In [85]:
#Change data type for age to integer
df['age'] = df['age'].astype(int)

In [86]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 594643 entries, 0 to 594642
Data columns (total 10 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   step         594643 non-null  int64  
 1   customer     594643 non-null  object 
 2   age          594643 non-null  int32  
 3   gender       594643 non-null  object 
 4   zipcodeOri   594643 non-null  object 
 5   merchant     594643 non-null  object 
 6   zipMerchant  594643 non-null  object 
 7   category     594643 non-null  object 
 8   amount       594643 non-null  float64
 9   fraud        594643 non-null  bool   
dtypes: bool(1), float64(1), int32(1), int64(1), object(6)
memory usage: 39.1+ MB


In [87]:
df.count()

step           594643
customer       594643
age            594643
gender         594643
zipcodeOri     594643
merchant       594643
zipMerchant    594643
category       594643
amount         594643
fraud          594643
dtype: int64

Change data type zipcodeOri and zipMerchant to integer

In [88]:
df['zipcodeOri'] = df['zipcodeOri'].astype(int)
df['zipMerchant'] = df['zipMerchant'].astype(int)

In [89]:
df.dtypes

step             int64
customer        object
age              int32
gender          object
zipcodeOri       int32
merchant        object
zipMerchant      int32
category        object
amount         float64
fraud             bool
dtype: object

Casefolding : Transform data in the columns customer, gender, and merchant into lowercase

In [90]:
df['customer'] = df['customer'].str.lower()
df['gender'] = df['gender'].str.lower()
df['merchant'] = df['merchant'].str.lower()

Change columns name, zipcodeOri to zipcodeori and zipMerchant to zipmerchant (lowercase)

In [91]:
df.rename(columns={"zipcodeOri": "zipcodeori", "zipMerchant": "zipmerchant"}, inplace=True)

In [92]:
df.head()

,step,customer,age,gender,zipcodeori,merchant,zipmerchant,category,amount,fraud
0,0,c1093826151,4,m,28007,m348934600,28007,es_transportation,4.55,False
1,0,c352968107,2,m,28007,m348934600,28007,es_transportation,39.68,False
2,0,c2054744914,4,f,28007,m1823072687,28007,es_transportation,26.89,False
3,0,c1760612790,3,m,28007,m348934600,28007,es_transportation,17.25,False
4,0,c757503768,5,m,28007,m348934600,28007,es_transportation,35.72,False


Check null

In [93]:
if df.isnull().values.any():
    print("Ada nilai null")
else:
    print("Tidak ada nilai null")

Tidak ada nilai null


In [94]:
df.isna()

,step,customer,age,gender,zipcodeori,merchant,zipmerchant,category,amount,fraud
0,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...
594638,False,False,False,False,False,False,False,False,False,False
594639,False,False,False,False,False,False,False,False,False,False
594640,False,False,False,False,False,False,False,False,False,False
594641,False,False,False,False,False,False,False,False,False,False


In [95]:
df.count()

step           594643
customer       594643
age            594643
gender         594643
zipcodeori     594643
merchant       594643
zipmerchant    594643
category       594643
amount         594643
fraud          594643
dtype: int64

In [96]:
df.head()

,step,customer,age,gender,zipcodeori,merchant,zipmerchant,category,amount,fraud
0,0,c1093826151,4,m,28007,m348934600,28007,es_transportation,4.55,False
1,0,c352968107,2,m,28007,m348934600,28007,es_transportation,39.68,False
2,0,c2054744914,4,f,28007,m1823072687,28007,es_transportation,26.89,False
3,0,c1760612790,3,m,28007,m348934600,28007,es_transportation,17.25,False
4,0,c757503768,5,m,28007,m348934600,28007,es_transportation,35.72,False


Save dataframe to csv file

In [54]:
df.to_csv("bs_clean.csv", index=False, header=False)

In [30]:
#df.to_csv("bsclean.csv", index=False)

TABLE 2 (bsnet140513_032310)

CREATE TABLE

In [31]:
create_table_query2 = '''CREATE TABLE bsnet140513_032310 (
    Source varchar (20),
    Target varchar (20),
    Weight float4,
    typeTrans varchar (30),
    fraud boolean
); '''


try:
    # Make connection to db
    cxn2 = psycopg2.connect(CONNECT_DB)

    # Create a cursor to db
    cur2 = cxn2.cursor()
    
    # Send sql query to request
    cur2.execute(create_table_query2)
    records2 = cxn2.commit()

except (Exception, psycopg2.Error) as error :
    print ("Error while connecting to PostgreSQL", error)
    
finally:
    #closing database connection.
    if(cxn2):
        cur2.close()
        cxn2.close()
        print("PostgreSQL connection is closed")

print(f'Records:\n {records2}')

PostgreSQL connection is closed
Records:
 None


Add data to table 2

In [32]:
try:
    # Make connection to db
    cxn2 = psycopg2.connect(CONNECT_DB)
    
    # Create a cursor to db
    cur2 = cxn2.cursor()
    
    with open('./bsnet140513_032310.csv', 'r') as f:
        # skip first row, header row
        next(f)
        cur2.copy_from(f, 'bsnet140513_032310', sep=",")
        cxn2.commit()

except (Exception, psycopg2.Error) as error :
    print ("Error while connecting to PostgreSQL", error)
    
finally:
    #closing database connection.
    if(cxn2):
        cur2.close()
        cxn2.close()
        print("PostgreSQL connection is closed")
        print("bsnet140513_032310 table populated")


PostgreSQL connection is closed
bsnet140513_032310 table populated


Select data from server

In [33]:
#selecting data from server using fetchall()
def db_server_fetch(sql_query2):
    try:
        # Make connection to db
        cxn2 = psycopg2.connect(CONNECT_DB)

        # Create a cursor to db
        cur2 = cxn2.cursor()

        # Send sql query to request
        cur2.execute(sql_query2)
        records2 = cur2.fetchall()

    except (Exception, psycopg2.Error) as error :
        print ("Error while connecting to PostgreSQL", error)

    finally:
        #closing database connection.
        if(cxn2):
            cur2.close()
            cxn2.close()
            print("PostgreSQL connection is closed")
        return records2

In [34]:
select_query2 = '''SELECT * FROM bsnet140513_032310;'''

records2 = db_server_fetch(select_query2)

PostgreSQL connection is closed


Create dataframe

In [35]:
df2 = pd.DataFrame(records2, columns=["Source", "Target", "Weight", "typeTrans", "fraud"])

EDA (Exploratory Data Analysis) 

In [36]:
df2.head()

,Source,Target,Weight,typeTrans,fraud
0,'C1093826151','M348934600',4.55,'es_transportation',False
1,'C352968107','M348934600',39.68,'es_transportation',False
2,'C2054744914','M1823072687',26.89,'es_transportation',False
3,'C1760612790','M348934600',17.25,'es_transportation',False
4,'C757503768','M348934600',35.72,'es_transportation',False


In [44]:
df2.tail()

,Source,Target,Weight,typeTrans,fraud
594638,'C1753498738','M1823072687',20.53,'es_transportation',False
594639,'C650108285','M1823072687',50.73,'es_transportation',False
594640,'C123623130','M349281107',22.44,'es_fashion',False
594641,'C1499363341','M1823072687',14.46,'es_transportation',False
594642,'C616528518','M1823072687',26.93,'es_transportation',False


In [38]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 594643 entries, 0 to 594642
Data columns (total 5 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   Source     594643 non-null  object 
 1   Target     594643 non-null  object 
 2   Weight     594643 non-null  float64
 3   typeTrans  594643 non-null  object 
 4   fraud      594643 non-null  bool   
dtypes: bool(1), float64(1), object(3)
memory usage: 18.7+ MB


In [39]:
#Check null
if df2.isnull().values.any():
    print("Ada nilai null di df2")
else:
    print("Tidak ada nilai null di df2")

Tidak ada nilai null di df2


In [40]:
# to show all columns in `.describe` add `include="all"`
df2.describe(include="all")

,Source,Target,Weight,typeTrans,fraud
count,594643,594643,594643.000000,594643,594643
unique,4112,50,NaN,15,2
top,'C1978250683','M1823072687',NaN,'es_transportation',False
freq,265,299693,NaN,505119,587443
mean,NaN,NaN,37.890135,NaN,NaN
std,NaN,NaN,111.402831,NaN,NaN
min,NaN,NaN,0.000000,NaN,NaN
25%,NaN,NaN,13.740000,NaN,NaN
50%,NaN,NaN,26.900000,NaN,NaN
75%,NaN,NaN,42.540000,NaN,NaN


In [41]:
df2.describe()

,Weight
count,594643.000000
mean,37.890135
std,111.402831
min,0.000000
25%,13.740000
50%,26.900000
75%,42.540000
max,8329.960000


In [42]:
df2.count()

Source       594643
Target       594643
Weight       594643
typeTrans    594643
fraud        594643
dtype: int64

In [43]:
df2.dtypes

Source        object
Target        object
Weight       float64
typeTrans     object
fraud           bool
dtype: object

TRANSFORMASI DATA

Delete quotation mark in the columns : Source, Target, typeTrans

In [45]:
clean_columns_df2 = ['Source', 'Target', 'typeTrans']
for col in clean_columns_df2:
    df2[col] = df2[col].str.replace("'", "")

#Show 5 data
df2.head()

,Source,Target,Weight,typeTrans,fraud
0,C1093826151,M348934600,4.55,es_transportation,False
1,C352968107,M348934600,39.68,es_transportation,False
2,C2054744914,M1823072687,26.89,es_transportation,False
3,C1760612790,M348934600,17.25,es_transportation,False
4,C757503768,M348934600,35.72,es_transportation,False


Change columns name into lowercase : Source -> source, Target->targer, Weight -> weight, typeTrans->typetrans

In [48]:
df2.rename(columns={"Source": "source", "Target": "target", 'Weight': 'weight', 'typeTrans' : 'typetrans'}, inplace=True)

In [49]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 594643 entries, 0 to 594642
Data columns (total 5 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   source     594643 non-null  object 
 1   target     594643 non-null  object 
 2   weight     594643 non-null  float64
 3   typetrans  594643 non-null  object 
 4   fraud      594643 non-null  bool   
dtypes: bool(1), float64(1), object(3)
memory usage: 18.7+ MB


In [50]:
df2.head()

,source,target,weight,typetrans,fraud
0,C1093826151,M348934600,4.55,es_transportation,False
1,C352968107,M348934600,39.68,es_transportation,False
2,C2054744914,M1823072687,26.89,es_transportation,False
3,C1760612790,M348934600,17.25,es_transportation,False
4,C757503768,M348934600,35.72,es_transportation,False


Casefolding : Changes data in the source and target columns into lowercase

In [51]:
df2['source'] = df2['source'].str.lower()
df2['target'] = df2['target'].str.lower()

In [52]:
df2.head()

,source,target,weight,typetrans,fraud
0,c1093826151,m348934600,4.55,es_transportation,False
1,c352968107,m348934600,39.68,es_transportation,False
2,c2054744914,m1823072687,26.89,es_transportation,False
3,c1760612790,m348934600,17.25,es_transportation,False
4,c757503768,m348934600,35.72,es_transportation,False


In [97]:
if df2.isnull().values.any():
    print ("ada nilai null")
else:
    print ("tidak ada nilai null")

tidak ada nilai null


In [99]:
df2.count()

source       594643
target       594643
weight       594643
typetrans    594643
fraud        594643
dtype: int64

Save dataframe to csv file

In [53]:
df2.to_csv('bsnet_clean.csv', index=False, header=False)